# Example Notebook and Installation Check
This notebook provides an example of how to run an experiment with the `locomotion` package. It also acts as a check to make sure your package has been installed correctly.

---

## Import Check
First, we make sure that we are able to import the locomotion package.

In [ ]:
import sys
import os
PATH_TO_DIRECTORY = os.getcwd()

# If this works, then locomotion has been installed into your system.
import locomotion

## Data Setup
Next, we setup the data for our experiment, which involves writing `.json` files that captures the experimental data for each dataset. This can be done manually, as we've done below, or by using `scripts/json_generator.ipynb`. 

Note that 1 `.json` file needs to be generated for each experimental group, with a JSON item for each dataset in the group. In our example, we have 2 experimental groups with 2 datasets each, thus we create 2 `.json` files, each containing 2 JSON items.

In [ ]:
# No need to touch this
species = "medaka"
exp_type = "SS"
x_lims = (0, 200)
y_lims = (0, 100)
frames_per_sec = 20
pixels_per_mm = 1.6
start_time = 0
end_time = 2
baseline_start_time = 0
baseline_end_time = 1

import json
json_items_ss = [{
    "name": "SS_01",
    "data_file_location": PATH_TO_DIRECTORY + "/../samples/sample_data/SS_01.dat",
    "animal_attributes":
    {
      "species": species,
      "exp_type": exp_type,
      "ID": "01"
    },
    "capture_attributes": 
    {
      "x_lims": x_lims,
      "y_lims": y_lims,
      "pixels_per_mm": pixels_per_mm,
      "frames_per_sec": frames_per_sec,
      "start_time": start_time,
      "end_time": end_time,
      "baseline_start_time": baseline_start_time,
      "baseline_end_time": baseline_end_time
    },
    "additional_info": 
    {
      "point": [100, 100]
    }
  },
{
    "name": "SS_02",
    "data_file_location": PATH_TO_DIRECTORY + "/../samples/sample_data/SS_02.dat",
    "animal_attributes":
    {
      "species": species,
      "exp_type": exp_type,
      "ID": "02"
    },
    "capture_attributes": 
    {
      "x_lims": x_lims,
      "y_lims": y_lims,
      "pixels_per_mm": pixels_per_mm,
      "frames_per_sec": frames_per_sec,
      "start_time": start_time,
      "end_time": end_time,
      "baseline_start_time": baseline_start_time,
      "baseline_end_time": baseline_end_time
    },
    "additional_info": 
    {
      "point": [50, 50]
    }
  }]
json_items_nss = [{
    "name": "NSS_01",
    "data_file_location": PATH_TO_DIRECTORY + "/../samples/sample_data/NSS_01.dat",
    "animal_attributes":
    {
      "species": species,
      "exp_type": exp_type,
      "ID": "01"
    },
    "capture_attributes": 
    {
      "x_lims": x_lims,
      "y_lims": y_lims,
      "pixels_per_mm": pixels_per_mm,
      "frames_per_sec": frames_per_sec,
      "start_time": start_time,
      "end_time": end_time,
      "baseline_start_time": baseline_start_time,
      "baseline_end_time": baseline_end_time
    },
    "additional_info": 
    {
      "point": [30, 30]
    }
  },
{
    "name": "NSS_02",
    "data_file_location": PATH_TO_DIRECTORY + "/../samples/sample_data/NSS_02.dat",
    "animal_attributes":
    {
      "species": species,
      "exp_type": exp_type,
      "ID": "02"
    },
    "capture_attributes": 
    {
      "x_lims": x_lims,
      "y_lims": y_lims,
      "pixels_per_mm": pixels_per_mm,
      "frames_per_sec": frames_per_sec,
      "start_time": start_time,
      "end_time": end_time,
      "baseline_start_time": baseline_start_time,
      "baseline_end_time": baseline_end_time
    },
    "additional_info": 
    {
      "point": [10, 10]
    }
  }]

outfile_ss = PATH_TO_DIRECTORY + "/../samples/sample_check_SS.json"
outfile_nss = PATH_TO_DIRECTORY + "/../samples/sample_check_NSS.json"
jsonstr_ss = json.dumps(json_items_ss, indent=4)
jsonstr_nss = json.dumps(json_items_nss, indent=4)
with open(outfile_ss, "w") as outfile:
    outfile.write(jsonstr_ss)
    print("SS information wrote to %s" % outfile_ss)
with open(outfile_nss, "w") as outfile:
    outfile.write(jsonstr_nss)
    print("NSS information wrote to %s" % outfile_nss)

## Trajectory Methods
This section will run through the 2 main methods provided in `trajectory.py`:
* `trajectory.compute_all_bdd()` computes the pair-wise Behavioral Distortion Distance (BDD) between all animals in the experiment,
* `trajectory.compute_all_iibdd()` computes the Intra-Individual BDD of each animal in the experiment, by computing the BDD of an animal to itself over a pair of non-overlapping time intervals of equal length.

In [ ]:
info_files = [outfile_ss, outfile_nss]
animals = locomotion.setup_animal_objs( info_files )
for a in animals:
    first_deriv, velocity = locomotion.trajectory.populate_velocity( a )
    locomotion.trajectory.populate_curvature(a, first_deriv=first_deriv, velocity=velocity)
    locomotion.trajectory.populate_distance_from_point(a, "point", 'Dist to Point', col_names=['X', 'Y'])
    

In [ ]:
# Testing BDD methods
variables = ['Velocity','Curvature', 'Dist to Point']
norm_mode = 'spec'
number_of_comparisons_per_animal, specified_durations = 100, None
start_time, end_time = 0, 1
distances = locomotion.trajectory.compute_all_bdd(animals,
                                                variables,
                                                start_time,
                                                end_time,
                                                norm_mode)


In [ ]:
output_directory, outfile_name = PATH_TO_DIRECTORY + "/results", "check_BDD"
try: # Safety check to ensure that data folder exists, and makes it otherwise.
    os.mkdir(output_directory)
except FileExistsError:
    pass
sort_table, square_table = False, False
color_min, color_max = 0.1, 0.2
locomotion.write.post_process(animals,
                             distances,
                             output_directory,
                             outfile_name,
                             sort_table,
                             square_table,
                             color_min,
                             color_max)

In [ ]:
print(distances)
# Print result should be similar to the following:
# [['', 0.13316423888910794, 0.1479803274285245, 0.1514418903180464], 
# ['', '', 0.15586433304624217, 0.13717273747247993], 
# ['', '', '', 0.1608370955689277], 
# ['', '', '', '']]

In [ ]:
# Testing IIBDD Methods
variables = ['Y','Velocity','Curvature']
norm_mode = 'spec'
num_samples = 5
iibdds = locomotion.trajectory.compute_all_iibdd(animals, variables, norm_mode, num_samples)

In [ ]:
output_directory, outfile_name = PATH_TO_DIRECTORY + "/results", "check_IIBDD.csv"
try: # Safety check to ensure that data folder exists, and makes it otherwise.
    os.mkdir(output_directory)
except FileExistsError:
    pass
locomotion.write.write_iibdd_to_csv(animals, iibdds, output_directory, outfile_name)

## Heatmap Methods
This section will run through the main method provided in `heatmap.py`, namely `heatmap.compute_all_csd()`, which calculates the pairwise Conformal Spatiotemporal Distance (CSD) between all animals in the experiment. 

In [ ]:
grid_size, start_time, end_time = 10, 0, 2
for a in animals:
    locomotion.heatmap.populate_surface_data(a, grid_size, start_time, end_time)

In [ ]:
distances = locomotion.heatmap.compute_all_csd(animals)

In [ ]:
output_directory, outfile_name = PATH_TO_DIRECTORY + "/results", "results"
sort_table, square_table = False, False
color_min, color_max = 0, 0.02
locomotion.write.post_process(animals,
                             distances,
                             output_directory,
                             outfile_name,
                             sort_table,
                             square_table,
                             color_min,
                             color_max)

# Once you are done, please remember to clear all outputs with 'Cell > All Output > Clear'.